In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Rainfall Temperature").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/unit-7/rainfall_temperature.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("rainfall_temperature.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+----------------+--------------------+-------------------+-------------------+-------------------+----------+--------------+-------------------+----+------+----------+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+----+
|OrganisationName|    OrganisationCode|      PublishedDate|       DurationFrom|         DurationTo|LatestData|       GeoName|ReportingPeriodType|Year|Period|Rainfallmm| Temp|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32| _c33|_c34|_c35|
+----------------+--------------------+-------------------+-------------------+-------------------+----------+--------------+-------------------+----+------+----------+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+----+----+
|    Lincolnshire|http://opendataco...|2019-02-27T00:00:00|2010-01-01T00:00:00|2010-01-31T00:00:00|      TRUE|En

In [4]:
def celsius_to_fahrenheit(temperature):
  temperature = float(temperature)
  temperature = (temperature * 1.8) + 32
  return temperature

In [5]:
spark.udf.register("celsius_to_fahrenheit", celsius_to_fahrenheit)

<function __main__.celsius_to_fahrenheit>

In [ ]:
df.createOrReplaceTempView("rainfall_temperature")

In [6]:
temp_df = spark.sql("SELECT PublishedDate, celsius_to_fahrenheit(Temp) FROM rainfall_temperature")
temp_df.show()

+-------------------+---------------------------+
|      PublishedDate|celsius_to_fahrenheit(Temp)|
+-------------------+---------------------------+
|2019-02-27T00:00:00|                      33.44|
|2019-02-27T00:00:00|                      34.88|
|2019-02-27T00:00:00|                       41.9|
|2019-02-27T00:00:00|                      46.76|
|2019-02-27T00:00:00|                      49.64|
|2019-02-27T00:00:00|                      57.74|
|2019-02-27T00:00:00|         61.519999999999996|
|2019-02-27T00:00:00|                      58.28|
|2019-02-27T00:00:00|         56.120000000000005|
|2019-02-27T00:00:00|                      49.28|
|2019-02-27T00:00:00|                       40.1|
|2019-02-27T00:00:00|                      29.84|
|2019-02-27T00:00:00|                     34.736|
|2019-02-27T00:00:00|         46.129999999999995|
|2019-02-27T00:00:00|         59.144000000000005|
|2019-02-27T00:00:00|                     48.506|
|2019-02-27T00:00:00|                     46.688|
